In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import wrds
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

### deal with the crsp data

In [4]:
# 从本地文件中读取数据
crsp = pd.read_csv('data/weighted_return.zip', 
                   parse_dates=['date'], infer_datetime_format=True)

In [5]:
# 预览数据
crsp.head()

,PERMNO,date,TICKER,COMNAM,CUSIP,PRC,RET,SHROUT,RETX,vwretd,vwretx,ewretd,ewretx
0,10000,1985-12-31,NaN,NaN,68391610,NaN,NaN,NaN,NaN,0.043061,0.040080,0.028021,0.026355
1,10000,1986-01-31,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-4.3750,C,3680.0,C,0.009829,0.008006,0.044071,0.043082
2,10000,1986-02-28,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-3.2500,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938
3,10000,1986-03-31,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-4.4375,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679
4,10000,1986-04-30,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-4.0000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141


In [6]:
# 将PERMNO的数据类型转换成integer
crsp[['PERMNO']] = crsp[['PERMNO']].astype(int)

In [11]:
# 替换RET, RETX中的B, C为np.nan, 并且将所有的np.nan填充为0
crsp.RET = crsp.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
crsp.RETX = crsp.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)

crsp.RET = crsp.RET.fillna(0)
crsp.RETX = crsp.RETX.fillna(0)

In [12]:
# 预览数据
crsp.head()

,PERMNO,date,TICKER,COMNAM,CUSIP,PRC,RET,SHROUT,RETX,vwretd,vwretx,ewretd,ewretx
0,10000,1985-12-31,NaN,NaN,68391610,NaN,0.000000,NaN,0.000000,0.043061,0.040080,0.028021,0.026355
1,10000,1986-01-31,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-4.3750,0.000000,3680.0,0.000000,0.009829,0.008006,0.044071,0.043082
2,10000,1986-02-28,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-3.2500,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938
3,10000,1986-03-31,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-4.4375,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679
4,10000,1986-04-30,OMFGA,OPTIMUM MANUFACTURING INC,68391610,-4.0000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141


In [13]:
# 将多余的列去掉
crsp = crsp.drop(['CUSIP', 'TICKER', 'COMNAM'], axis=1)

In [14]:
# 预览数据
crsp.head()

,PERMNO,date,PRC,RET,SHROUT,RETX,vwretd,vwretx,ewretd,ewretx
0,10000,1985-12-31,NaN,0.000000,NaN,0.000000,0.043061,0.040080,0.028021,0.026355
1,10000,1986-01-31,-4.3750,0.000000,3680.0,0.000000,0.009829,0.008006,0.044071,0.043082
2,10000,1986-02-28,-3.2500,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938
3,10000,1986-03-31,-4.4375,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679
4,10000,1986-04-30,-4.0000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141


In [15]:
# 将数据的列名称转换成小写英文字符
crsp.columns = [i.lower() for i in crsp.columns]
crsp.head()

,permno,date,prc,ret,shrout,retx,vwretd,vwretx,ewretd,ewretx
0,10000,1985-12-31,NaN,0.000000,NaN,0.000000,0.043061,0.040080,0.028021,0.026355
1,10000,1986-01-31,-4.3750,0.000000,3680.0,0.000000,0.009829,0.008006,0.044071,0.043082
2,10000,1986-02-28,-3.2500,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938
3,10000,1986-03-31,-4.4375,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679
4,10000,1986-04-30,-4.0000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141


In [16]:
# 计算市值, RET+1, RETX+1
crsp = crsp.assign(mv = crsp.prc.abs() * crsp.shrout,
                   ret_p1 = crsp.ret+1,
                   retx_p1 = crsp.retx+1,
                   vwretd_p1 = crsp.vwretd+1,
                   vwretx_p1 = crsp.vwretx+1,
                   ewretd_p1 = crsp.ewretd+1,
                   ewretx_p1 = crsp.ewretx+1)

In [17]:
# 预览数据
crsp.head()

,permno,date,prc,ret,shrout,retx,vwretd,vwretx,ewretd,ewretx,mv,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1
0,10000,1985-12-31,NaN,0.000000,NaN,0.000000,0.043061,0.040080,0.028021,0.026355,NaN,1.000000,1.000000,1.043061,1.040080,1.028021,1.026355
1,10000,1986-01-31,-4.3750,0.000000,3680.0,0.000000,0.009829,0.008006,0.044071,0.043082,16100.0,1.000000,1.000000,1.009829,1.008006,1.044071,1.043082
2,10000,1986-02-28,-3.2500,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938,11960.0,0.742857,0.742857,1.072500,1.068190,1.060381,1.058938
3,10000,1986-03-31,-4.4375,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679,16330.0,1.365385,1.365385,1.053885,1.051360,1.047192,1.045679
4,10000,1986-04-30,-4.0000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141,15172.0,0.901408,0.901408,0.992097,0.990367,1.016140,1.015141


In [14]:
for i in range(11):
    print(i)

0
1
2
3
4
5
6
7
8
9
10


In [18]:
# 查看各列在剔除异常值的前后的数据行数
for i in range(len(crsp.columns)):
    print(crsp.columns[i], crsp.iloc[:,i].shape[0], crsp.iloc[:,i].dropna().shape[0])

permno 3395194 3395194
date 3395194 3395194
prc 3395194 3276882
ret 3395194 3395194
shrout 3395194 3369390
retx 3395194 3395194
vwretd 3395194 3395194
vwretx 3395194 3395194
ewretd 3395194 3395194
ewretx 3395194 3395194
mv 3395194 3276882
ret_p1 3395194 3395194
retx_p1 3395194 3395194
vwretd_p1 3395194 3395194
vwretx_p1 3395194 3395194
ewretd_p1 3395194 3395194
ewretx_p1 3395194 3395194


In [18]:
# 剔除异常值 亦可以使用subset参数
crsp = crsp.dropna()

In [19]:
# 预览数据
crsp.head()

,permno,date,prc,ret,shrout,retx,vwretd,vwretx,ewretd,ewretx,mv,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1
1,10000,1986-01-31,-4.37500,0.000000,3680.0,0.000000,0.009829,0.008006,0.044071,0.043082,16100.00000,1.000000,1.000000,1.009829,1.008006,1.044071,1.043082
2,10000,1986-02-28,-3.25000,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938,11960.00000,0.742857,0.742857,1.072500,1.068190,1.060381,1.058938
3,10000,1986-03-31,-4.43750,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679,16330.00000,1.365385,1.365385,1.053885,1.051360,1.047192,1.045679
4,10000,1986-04-30,-4.00000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141,15172.00000,0.901408,0.901408,0.992097,0.990367,1.016140,1.015141
5,10000,1986-05-30,-3.10938,-0.222656,3793.0,-0.222656,0.050844,0.047123,0.036194,0.034779,11793.87834,0.777344,0.777344,1.050844,1.047123,1.036194,1.034779


In [20]:
# 查看当前有多少行数据
crsp.shape[0]

3276882

In [21]:
# 确保date为每月最后一天
crsp.date = pd.to_datetime(crsp.date) + MonthEnd(0)

In [22]:
# 可以对比第5条数据的date,处理之前是1986-05-30, 处理之后是1986-05-31
crsp.head()

,permno,date,prc,ret,shrout,retx,vwretd,vwretx,ewretd,ewretx,mv,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1
1,10000,1986-01-31,-4.37500,0.000000,3680.0,0.000000,0.009829,0.008006,0.044071,0.043082,16100.00000,1.000000,1.000000,1.009829,1.008006,1.044071,1.043082
2,10000,1986-02-28,-3.25000,-0.257143,3680.0,-0.257143,0.072500,0.068190,0.060381,0.058938,11960.00000,0.742857,0.742857,1.072500,1.068190,1.060381,1.058938
3,10000,1986-03-31,-4.43750,0.365385,3680.0,0.365385,0.053885,0.051360,0.047192,0.045679,16330.00000,1.365385,1.365385,1.053885,1.051360,1.047192,1.045679
4,10000,1986-04-30,-4.00000,-0.098592,3793.0,-0.098592,-0.007903,-0.009633,0.016140,0.015141,15172.00000,0.901408,0.901408,0.992097,0.990367,1.016140,1.015141
5,10000,1986-05-31,-3.10938,-0.222656,3793.0,-0.222656,0.050844,0.047123,0.036194,0.034779,11793.87834,0.777344,0.777344,1.050844,1.047123,1.036194,1.034779


In [23]:
# 读取cpi数据，并对mv做调整
cpi = pd.read_excel('data/CPIAUCSL.xlsx')

In [24]:
cpi.head()

,date,cpiaucsl,cpi_adjust
0,1973-01-01,42.7,0.201250
1,1973-02-01,43.0,0.202175
2,1973-03-01,43.4,0.203328
3,1973-04-01,43.7,0.204261
4,1973-05-01,43.9,0.203989


In [26]:
# MonthEnd(-1)的效果
(pd.to_datetime(cpi.date) + MonthEnd(-1)).head()

0   1972-12-31
1   1973-01-31
2   1973-02-28
3   1973-03-31
4   1973-04-30
Name: date, dtype: datetime64[ns]

In [25]:
cpi.date = pd.to_datetime(cpi.date) + MonthEnd(-1)

In [26]:
# 做了MonthEnd(-1之后的效果)
cpi.head()

,date,cpiaucsl,cpi_adjust
0,1972-12-31,42.7,0.201250
1,1973-01-31,43.0,0.202175
2,1973-02-28,43.4,0.203328
3,1973-03-31,43.7,0.204261
4,1973-04-30,43.9,0.203989


In [27]:
# join the table
crsp = pd.merge(crsp, cpi[['date', 'cpi_adjust']], on=['date'])

In [28]:
crsp.head()

,permno,date,prc,ret,shrout,retx,vwretd,vwretx,ewretd,ewretx,mv,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1,cpi_adjust
0,10000,1986-01-31,-4.375,0.0,3680.0,0.0,0.009829,0.008006,0.044071,0.043082,16100.000,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781
1,10001,1986-01-31,-6.125,0.0,985.0,0.0,0.009829,0.008006,0.044071,0.043082,6033.125,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781
2,10002,1986-01-31,-11.625,0.0,1175.0,0.0,0.009829,0.008006,0.044071,0.043082,13659.375,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781
3,10003,1986-01-31,-22.000,0.0,1900.0,0.0,0.009829,0.008006,0.044071,0.043082,41800.000,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781
4,10005,1986-01-31,-0.375,0.0,4655.0,0.0,0.009829,0.008006,0.044071,0.043082,1745.625,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781


In [29]:
crsp = crsp.assign(mv_adj=crsp.mv / crsp.cpi_adjust)

In [30]:
crsp.head()

,permno,date,prc,ret,shrout,retx,vwretd,vwretx,ewretd,ewretx,mv,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1,cpi_adjust,mv_adj
0,10000,1986-01-31,-4.375,0.0,3680.0,0.0,0.009829,0.008006,0.044071,0.043082,16100.000,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781,31214.773929
1,10001,1986-01-31,-6.125,0.0,985.0,0.0,0.009829,0.008006,0.044071,0.043082,6033.125,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781,11697.057948
2,10002,1986-01-31,-11.625,0.0,1175.0,0.0,0.009829,0.008006,0.044071,0.043082,13659.375,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781,26482.875940
3,10003,1986-01-31,-22.000,0.0,1900.0,0.0,0.009829,0.008006,0.044071,0.043082,41800.000,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781,81042.083865
4,10005,1986-01-31,-0.375,0.0,4655.0,0.0,0.009829,0.008006,0.044071,0.043082,1745.625,1.0,1.0,1.009829,1.008006,1.044071,1.043082,0.515781,3384.427934


In [33]:
crsp.columns

Index(['permno', 'date', 'prc', 'ret', 'shrout', 'retx', 'vwretd', 'vwretx',
       'ewretd', 'ewretx', 'mv', 'ret_p1', 'retx_p1', 'vwretd_p1', 'vwretx_p1',
       'ewretd_p1', 'ewretx_p1', 'cpi_adjust', 'mv_adj'],
      dtype='object')

In [31]:
crsp = crsp[['date', 'permno', 'mv', 'mv_adj', 'ret_p1', 'retx_p1', 'vwretd_p1', 
             'vwretx_p1', 'ewretd_p1', 'ewretx_p1']]

In [32]:
crsp.head()

,date,permno,mv,mv_adj,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1
0,1986-01-31,10000,16100.000,31214.773929,1.0,1.0,1.009829,1.008006,1.044071,1.043082
1,1986-01-31,10001,6033.125,11697.057948,1.0,1.0,1.009829,1.008006,1.044071,1.043082
2,1986-01-31,10002,13659.375,26482.875940,1.0,1.0,1.009829,1.008006,1.044071,1.043082
3,1986-01-31,10003,41800.000,81042.083865,1.0,1.0,1.009829,1.008006,1.044071,1.043082
4,1986-01-31,10005,1745.625,3384.427934,1.0,1.0,1.009829,1.008006,1.044071,1.043082


In [33]:
# 整合decile return到crsp中去
decile_return = pd.read_csv('data/decile_return.zip',
                           parse_dates=['date'], infer_datetime_format=True)

In [34]:
# 预览数据
decile_return.head()

,permno,ret,date,capn,decret
0,10006.0,-0.060367,1973-01-31,9.0,-0.054701
1,10006.0,-0.075978,1973-02-28,9.0,-0.069455
2,10006.0,0.110429,1973-03-31,9.0,-0.024459
3,10006.0,-0.002762,1973-04-30,9.0,-0.072945
4,10006.0,-0.011634,1973-05-31,9.0,-0.059616


In [35]:
# 将无效值填充为0
decile_return.decret = decile_return.decret.fillna(0)

In [36]:
# 整合decile return到crsp中去
crsp = pd.merge(crsp, decile_return[['date', 'permno', 'decret']], 
                 on=['date', 'permno'])

In [37]:
# 计算decile return + 1
crsp = crsp.assign(dret_p1 = crsp.decret + 1)
crsp = crsp.drop('decret', axis=1)

In [38]:
crsp.head()

,date,permno,mv,mv_adj,ret_p1,retx_p1,vwretd_p1,vwretx_p1,ewretd_p1,ewretx_p1,dret_p1
0,1986-01-31,10000,16100.000,31214.773929,1.0,1.0,1.009829,1.008006,1.044071,1.043082,1.051443
1,1986-01-31,10001,6033.125,11697.057948,1.0,1.0,1.009829,1.008006,1.044071,1.043082,1.062222
2,1986-01-31,10002,13659.375,26482.875940,1.0,1.0,1.009829,1.008006,1.044071,1.043082,1.066751
3,1986-01-31,10003,41800.000,81042.083865,1.0,1.0,1.009829,1.008006,1.044071,1.043082,1.030587
4,1986-01-31,10005,1745.625,3384.427934,1.0,1.0,1.009829,1.008006,1.044071,1.043082,1.086720


In [39]:
# 把合成好的数据存储到本地
crsp.to_csv('data/return_ensembled.csv', index=False)

数据保存后，比较大的数据，我会用7z这类软件把csv转成zip格式,后续读取的数据都是读取的zip格式的同名文件

### deal with the comp & ccm link

In [59]:
comp = pd.read_csv('data/fundamental 1973-2013.zip',
                  parse_dates=['datadate'], infer_datetime_format=True)

In [60]:
comp.datadate = pd.to_datetime(comp.datadate) + MonthEnd(0)

In [61]:
comp.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,dlto,dltt,ebit,ib,pstk,revt,xrd,costat,cshtr_f,fyrc
0,1000,1973-12-31,1973.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,0.000,7.000,3.277,1.863,0.000,37.750,NaN,I,433800.0,12
1,1000,1974-12-31,1974.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,0.000,7.000,3.494,1.555,0.414,50.325,NaN,I,185600.0,12
2,1000,1975-12-31,1975.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,2.092,4.592,5.335,2.284,2.069,51.192,NaN,I,353100.0,12
3,1000,1976-12-31,1976.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,9.133,13.583,7.143,3.434,1.569,66.414,NaN,I,944700.0,12
4,1000,1977-12-31,1977.0,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,9.500,18.116,3.503,1.928,0.000,77.946,NaN,I,1228900.0,12


In [62]:
# drop不需要的列
comp = comp.drop(['indfmt', 'consol', 'popsrc', 'datafmt', 'cusip'], axis=1)

In [63]:
comp[comp.fyr.isna()].head()

,gvkey,datadate,fyear,tic,conm,curcd,fyr,bkvlps,dlto,dltt,ebit,ib,pstk,revt,xrd,costat,cshtr_f,fyrc
17755,2109,2011-12-31,NaN,CHSO,CHINA SHOUGUAN MINING CORP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,12
57476,4606,2009-12-31,NaN,COLE,COLE REAL ESTATE INVESTMENTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I,NaN,12
91236,6683,2011-12-31,NaN,STSC,START SCIENTIFIC INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,12
151746,10469,2011-10-31,NaN,CHEXF,AVIVAGEN INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,10
172254,11756,2007-12-31,NaN,ALEX,ALEXANDER & BALDWIN INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,12


In [64]:
# 剔除掉fyr中的异常值
comp = comp[comp.fyr > 0]

In [65]:
comp.head()

,gvkey,datadate,fyear,tic,conm,curcd,fyr,bkvlps,dlto,dltt,ebit,ib,pstk,revt,xrd,costat,cshtr_f,fyrc
0,1000,1973-12-31,1973.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,3.0165,0.000,7.000,3.277,1.863,0.000,37.750,NaN,I,433800.0,12
1,1000,1974-12-31,1974.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,4.5781,0.000,7.000,3.494,1.555,0.414,50.325,NaN,I,185600.0,12
2,1000,1975-12-31,1975.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,4.8808,2.092,4.592,5.335,2.284,2.069,51.192,NaN,I,353100.0,12
3,1000,1976-12-31,1976.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,7.1024,9.133,13.583,7.143,3.434,1.569,66.414,NaN,I,944700.0,12
4,1000,1977-12-31,1977.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,7.1321,9.500,18.116,3.503,1.928,0.000,77.946,NaN,I,1228900.0,12


In [17]:
datetime.datetime.today().strftime('%b')

'Mar'

In [66]:
# 为了进行fiscal resample, 将fiscal end month数字形式转化成'A-Dec'这样的字符串形式
# 这么做是方便在pandas下去做fiscal resample
# fysm -- fiscal year start month 财务年度起始日期, 即从datadate往前推12个月
comp = comp.assign(fystr = comp.datadate.apply(lambda x :  'A-' + x.strftime('%b')),
                  fysm = comp.datadate - MonthBegin(12))

In [67]:
# 可以看下datadate和fysm之间的关系, fystr是'A-'和月份英文简写
comp.head()

,gvkey,datadate,fyear,tic,conm,curcd,fyr,bkvlps,dlto,dltt,ebit,ib,pstk,revt,xrd,costat,cshtr_f,fyrc,fystr,fysm
0,1000,1973-12-31,1973.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,3.0165,0.000,7.000,3.277,1.863,0.000,37.750,NaN,I,433800.0,12,A-Dec,1973-01-01
1,1000,1974-12-31,1974.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,4.5781,0.000,7.000,3.494,1.555,0.414,50.325,NaN,I,185600.0,12,A-Dec,1974-01-01
2,1000,1975-12-31,1975.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,4.8808,2.092,4.592,5.335,2.284,2.069,51.192,NaN,I,353100.0,12,A-Dec,1975-01-01
3,1000,1976-12-31,1976.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,7.1024,9.133,13.583,7.143,3.434,1.569,66.414,NaN,I,944700.0,12,A-Dec,1976-01-01
4,1000,1977-12-31,1977.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,7.1321,9.500,18.116,3.503,1.928,0.000,77.946,NaN,I,1228900.0,12,A-Dec,1977-01-01


In [47]:
# 读取ccm link数据
ccm = pd.read_csv('data/ccm_hist.csv')

In [48]:
ccm.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,linkdt,linkenddt
0,1000,C,00X,NU,NaN,NaN,1961/1/1,1970/9/29
1,1000,P,1,NU,NaN,NaN,1970/9/30,1970/11/12
2,1000,P,1,LU,25881.0,23369.0,1970/11/13,1978/6/30
3,1001,C,00X,NU,NaN,NaN,1978/1/1,1983/9/19
4,1001,P,1,LU,10015.0,6398.0,1983/9/20,1986/7/31


In [49]:
# 选取特定的linktype的数据, 并且保持gvkey的唯一性
ccm = ccm[((ccm.linktype == 'LU') | (ccm.linktype == 'LC')) & \
   ((ccm.linkprim == 'P') | (ccm.linkprim == 'C'))]

In [50]:
ccm.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,linkdt,linkenddt
2,1000,P,1,LU,25881.0,23369.0,1970/11/13,1978/6/30
4,1001,P,1,LU,10015.0,6398.0,1983/9/20,1986/7/31
8,1002,C,1,LC,10023.0,22159.0,1972/12/14,1973/6/5
11,1003,C,1,LU,10031.0,6672.0,1983/12/7,1989/8/16
14,1004,P,1,LU,54594.0,20000.0,1972/4/24,NaN


In [51]:
ccm.gvkey.shape[0]

29210

In [52]:
ccm.gvkey.unique().shape[0]

25643

In [68]:
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# 将linkenddt缺省值设置为当天的日期
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm.columns = ['gvkey', 'linkprim', 'liid', 'linktype', 'permno', 'permco', 'linkdt',
       'linkenddt']
ccm1 = ccm[['gvkey', 'permno', 'linkdt', 'linkenddt']]
ccm1.permno = ccm1.permno.astype(int)

E:\ProgramData\Anaconda3\envs\quant\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [69]:
ccm1.head()

,gvkey,permno,linkdt,linkenddt
2,1000,25881,1970-11-13,1978-06-30 00:00:00.000000
4,1001,10015,1983-09-20,1986-07-31 00:00:00.000000
8,1002,10023,1972-12-14,1973-06-05 00:00:00.000000
11,1003,10031,1983-12-07,1989-08-16 00:00:00.000000
14,1004,54594,1972-04-24,2019-03-16 12:12:58.922897


In [70]:
comp.shape[0]

436885

In [71]:
# 将comp和ccm链接到一起
comp = pd.merge(left=comp, right=ccm1, on=['gvkey'])

In [72]:
comp.head()

,gvkey,datadate,fyear,tic,conm,curcd,fyr,bkvlps,dlto,dltt,...,revt,xrd,costat,cshtr_f,fyrc,fystr,fysm,permno,linkdt,linkenddt
0,1000,1973-12-31,1973.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,3.0165,0.000,7.000,...,37.750,NaN,I,433800.0,12,A-Dec,1973-01-01,25881,1970-11-13,1978-06-30
1,1000,1974-12-31,1974.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,4.5781,0.000,7.000,...,50.325,NaN,I,185600.0,12,A-Dec,1974-01-01,25881,1970-11-13,1978-06-30
2,1000,1975-12-31,1975.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,4.8808,2.092,4.592,...,51.192,NaN,I,353100.0,12,A-Dec,1975-01-01,25881,1970-11-13,1978-06-30
3,1000,1976-12-31,1976.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,7.1024,9.133,13.583,...,66.414,NaN,I,944700.0,12,A-Dec,1976-01-01,25881,1970-11-13,1978-06-30
4,1000,1977-12-31,1977.0,AE.2,A & E PLASTIK PAK INC,USD,12.0,7.1321,9.500,18.116,...,77.946,NaN,I,1228900.0,12,A-Dec,1977-01-01,25881,1970-11-13,1978-06-30


In [73]:
comp.shape[0]

424479

In [74]:
# 这里需要衡量到底要不要做这个筛选, 应该是要做筛选的，这样能保证不出错，但是数据会变少
comp[(comp.datadate>=comp.linkdt)&(comp.datadate<=comp.linkenddt)].shape[0]

276669

所以决定做两种，一种是不做筛选，一种做筛选

不做筛选

In [53]:
# 保留需要的列, 主要用于fiscal resample
comp1 = comp[['gvkey', 'datadate', 'fyear', 'fystr', 'fysm', 'permno']]

In [54]:
comp1.head()

,gvkey,datadate,fyear,fystr,fysm,permno
0,1000,1973-12-31,1973.0,A-Dec,1973-01-01,25881
1,1000,1974-12-31,1974.0,A-Dec,1974-01-01,25881
2,1000,1975-12-31,1975.0,A-Dec,1975-01-01,25881
3,1000,1976-12-31,1976.0,A-Dec,1976-01-01,25881
4,1000,1977-12-31,1977.0,A-Dec,1977-01-01,25881


In [55]:
# 把comp保存到本地, 用于后面fiscal resample
comp1.to_csv('data/fundamental_fiscal.csv', index=False)

另外一种是做筛选的

In [75]:
comp2 = comp[(comp.datadate>=comp.linkdt)&(comp.datadate<=comp.linkenddt)]
comp2 = comp2[['gvkey', 'datadate', 'fyear', 'fystr', 'fysm', 'permno']]
comp2.to_csv('data/fundamental_fiscal_filtered.csv', index=False)